In [ ]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis= 1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados [["preco", "idade_do_modelo", "km_por_ano"]]
y = dados ["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [ ]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 50.96%


/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


In [ ]:
  # Testando com outro SEED (p/ demonstrar a aleatoriedade)

x = dados [["preco", "idade_do_modelo", "km_por_ano"]]
y = dados ["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))
dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100
print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)
acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia do dummy stratified foi de 51.00%
A acurácia foi 76.84%


/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


O SEED, até aqui, foi aleatório e impacta muito no resultado. A decisão final seria tomada com base numa aleatoriedade

In [ ]:
from sklearn.model_selection import cross_validate 

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3, return_train_score=False)
results 

{'fit_time': array([0.01121593, 0.00683904, 0.00711322]),
 'score_time': array([0.00187135, 0.00179362, 0.00174093]),
 'test_score': array([0.75704859, 0.7629763 , 0.75337534])}

In [ ]:
results['test_score'] #Temos 3 notas pq o cv=3, treino e teste foram separados e rodaram 3x

array([0.75704859, 0.7629763 , 0.75337534])

In [ ]:
media=results['test_score'].mean()
media

0.7578000751484867

"Todavia, não é apenas a media que queremos, mas o intervalo. (...) sobre média e desvio padrão, veríamos que 95% da densidade estará dentro da média, mais ou menos dois desvios padrões."

`mean-2*std & mean+2*std`

In [ ]:
media=results['test_score'].mean()
desvio_padrao=results['test_score'].std()

print("Accuracy [%.2f, %.2f]" %((media-2*desvio_padrao)*100, (media+2*desvio_padrao)*100))

Accuracy [74.99, 76.57]


In [ ]:
#tudo junto
from sklearn.model_selection import cross_validate 

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media=results['test_score'].mean()
desvio_padrao=results['test_score'].std()
print("Accuracy com cross validation, 3 = [%.2f, %.2f]" %((media-2*desvio_padrao)*100, (media+2*desvio_padrao)*100))

Accuracy com cross validation, 3 = [75.21, 76.35]


O código acima já não está mais suscetível ao SEED. (bom!)

Porém, continua suscetível ao cv.

o cv default é 5. As documentações aconselham 5 ou 10.

**Como escolher o cv? quais os impactos?**

# **Aleatoriedade no Cross Validate**

In [ ]:
def imprimir_resultados(results):
  media = results ['test_score'].mean()
  desvio_padrao = results['test_score'].std()
  print("Accuracy médio: [%.2f]" %(media * 100))
  print("Accuracy intervalo: [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

In [ ]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)
cv = KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)  #usa aleatoriedade em casos específicos
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False) #é deterministico. E usa Kfold por baixo dos panos
#portanto, os resultados não estão sendo misturados antes de fazer o cross validation
imprimir_resultados(results)

Accuracy médio: [75.78]
Accuracy intervalo: [74.37, 77.19]


In [ ]:
#com o parâmetro de aleatoriedade do KFold

SEED = 301
np.random.seed(SEED)
cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)  #usa aleatoriedade em casos específicos
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False) #é deterministico. E usa Kfold por baixo dos panos
#portanto, os resultados não estão sendo misturados antes de fazer o cross validation
imprimir_resultados(results)

Accuracy médio: [75.76]
Accuracy intervalo: [73.26, 78.26]


"Na prática, se os dados não possuem alguma estrutura interna, como uma sequência por datas, por exemplo, primeiro gera-se um sequência aleatória desses dados e, em seguida, quebra-se em 10 pedaços ou faz-se ambos ao mesmo tempo."

# **Simular situação horrível de azar**

Pode ser "azar", como pode ser uma proporção de exemplos desbalanceados entre as classes.

In [ ]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [ ]:
#Sem shuffle
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)
cv = KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)  #usa aleatoriedade em casos específicos
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False) #é deterministico. E usa Kfold por baixo dos panos
#portanto, os resultados não estão sendo misturados antes de fazer o cross validation
imprimir_resultados(results)

Accuracy médio: [57.84]
Accuracy intervalo: [34.29, 81.39]


In [ ]:
#Com shuffle
SEED = 301
np.random.seed(SEED)
cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)  #usa aleatoriedade em casos específicos
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False) #é deterministico. E usa Kfold por baixo dos panos
#portanto, os resultados não estão sendo misturados antes de fazer o cross validation
imprimir_resultados(results)

Accuracy médio: [75.78]
Accuracy intervalo: [72.30, 79.26]


In [ ]:
#StratifiedKfold
#testar com e sem o shuffle
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)
cv = StratifiedKFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)  #usa aleatoriedade em casos específicos
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False) #é deterministico. E usa Kfold por baixo dos panos
#portanto, os resultados não estão sendo misturados antes de fazer o cross validation
imprimir_resultados(results)

Accuracy médio: [75.78]
Accuracy intervalo: [73.55, 78.01]


Aula4 (Dados agrupáveis): 
# Gerando dados aleatórios de modelos de carro para simulação de agrupamento ao usar nosso estimador


In [ ]:
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2,3, size=10000)
#substituir o negativo e somar 1 p/ não começar em 0
dados.modelo = dados.modelo+abs(dados.modelo.min())+1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [ ]:
dados.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

In [ ]:
dados.modelo.value_counts()

20    901
19    798
18    771
21    723
17    709
16    668
14    621
22    575
15    573
13    557
12    511
11    401
10    371
23    370
9     336
8     278
7     206
24    199
6     181
5     108
4      76
3      44
2      17
1       6
Name: modelo, dtype: int64

# Testando validação cruzada com GroupKFold

In [ ]:
from sklearn.model_selection import GroupKFold

SEED=301
np.random.seed(SEED)

cv=GroupKFold(n_splits= 10)
modelo=DecisionTreeClassifier(max_depth=2)
results=cross_validate(modelo, x_azar, y_azar, cv=cv, groups = dados.modelo,return_train_score=False)
imprimir_resultados(results)

Accuracy médio: [75.78]
Accuracy intervalo: [73.67, 77.90]


Quando as features tem ranges mt diferentes (ex.: 0 a 25 e 0 a 90000) isso impacta e aí precisamos fazer um pré-processamento 